<a href="https://colab.research.google.com/github/Amnahussain123-prog/Amnahussain123-prog/blob/main/LangChain_Rag_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install -Uq langchain-pinecone langchain-google-genai

In [32]:
from google.colab import userdata
import os
api_key="your_pinecone_api_key"
environment="your_pinecone_environment"
google_gemini_api_key="your_google_gemini_api_key"

api_key = userdata.get("PINECONE_API_KEY")
environment = os.getenv("PINECONE_ENVIRONMENT")
google_gemini_api_key = userdata.get("GOOGLE_API_KEY")

In [33]:
from pinecone import Pinecone, ServerlessSpec
api_key = userdata.get('PINECONE_API_KEY')
environment=os.getenv("PINECONE_ENVIRONMENT")

pc = Pinecone(
        api_key=userdata.get("PINECONE_API_KEY"),
        environment=os.getenv("PINECONE_ENVIRONMENT")
    )


In [34]:
!pip install  -qU --upgrade pinecone-client

In [130]:
import time

index_name = 'gemini-rag-index'
# Delete the existing index (if it exists)
try:
    pc.delete_index(index_name)
except:
    pass

# Create a new index with the correct dimension
pc.create_index(
    name=index_name,
    dimension=768,  # Change dimension to 768
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)
# Connect to the index
index = pc.Index(index_name)


In [131]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [132]:
vector = embeddings.embed_query("i'm a pre-medical student.")
vector[:5]

[-0.027932917699217796,
 -0.06260764598846436,
 0.012762178666889668,
 -0.026814168319106102,
 0.019282404333353043]

In [133]:
!pip install -Uq langchain-chroma
!pip install langchain_openai
!pip install -Uq langchain-core

In [134]:
import getpass
import os

from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [135]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [139]:
!pip install -Uq langchain_community

In [171]:
from langchain_core.documents import Document
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


docs_1 = Document (page_content=f"""
Pollution has tremendous adverse affects on environment, resulting in its degradation and also in depletion of living species. It has severe aftereffects like acid rain, global warming, famine, draught and extreme climatic conditions along with other consequences. The most common type of pollution is air pollution.
""", metadata={"source": "local"})

docs_2 = Document (page_content=f"""
Global Warming is a dangerous effect on our environment that we are facing these days. Rapid industrialization, increase in the population growth and pollution are causing a rise in Global Warming. Global Warming refers to the increase in the average temperature of the earth's surface during the last century.
""", metadata={"source": "local"})

docs_3 = Document (page_content= f"""
Artificial intelligence is a field of science concerned with building computers and machines that can reason, learn, and act in such a way that would normally require human intelligence or that involves data whose scale exceeds what humans can analyze.
""", metadata={"source": "local"})

docs_4 = Document(page_content=f"""
Agentic AI revolutionizes customer interactions by providing personalized and responsive experiences at scale and speed. Leveraging sophisticated models, AI agents can infer customer intent, predict needs, and offer tailored solutions, all while operating 24/7 to ensure consistent and efficient support.
""", metadata={"source": "local"})

docs_5 = Document(page_content=f"""
Retrieval-Augmented Generation (RAG) is the process of optimizing the output of a large language model, so it references an authoritative knowledge base outside of its training data sources before generating a response.
""", metadata={"Source": "local"})
loader = Document("documents.txt")  # Replace with your file

# Load documents
loader = TextLoader("documents.txt")  # Replace with your file
# DOCS = text_loader.load_documents([docs_1, docs_2, docs_3, docs_4, docs_5]) # text_loader is not defined, use loader instead, and it's unnecessary here
# DOCS = loader.load() # Uncomment this line if you want to load documents from a file

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
DOCS = text_splitter.split_documents([docs_1, docs_2, docs_3, docs_4, docs_5])

DOCS = [docs_1, docs_2, docs_3, docs_4, docs_5]

In [172]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

documents = [
    Document(page_content="your document content 1", metadata={"source": "local"}),
    Document(page_content="your document content 2", metadata={"source": "local"}),
    Document(page_content="your document content 3", metadata={"source": "local"}),
    Document(page_content="your document content 4", metadata={"source": "local"}),
    Document(page_content="your document content 5", metadata={"source": "local"})
]

# Pass the 'documents' list instead of the 'Document' class
vectorstore = Chroma.from_documents(
    documents,  # Pass the list of documents here
    embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001")
)

In [173]:
list(dir(vectorstore))

['_Chroma__ensure_collection',
 '_Chroma__query_collection',
 '_LANGCHAIN_DEFAULT_COLLECTION_NAME',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_asimilarity_search_with_relevance_scores',
 '_chroma_collection',
 '_client',
 '_client_settings',
 '_collection',
 '_collection_metadata',
 '_collection_name',
 '_cosine_relevance_score_fn',
 '_embedding_function',
 '_euclidean_relevance_score_fn',
 '_get_retriever_tags',
 '_max_inner_product_relevance_score_fn',
 '_persist_directory',
 '_select_relevance_score_fn',
 '_similarity_search_with_relevance_scores',
 'aadd_documents',
 'aadd_texts',
 'add_

In [175]:
from tqdm import tqdm
for doc in tqdm(DOCS):
    vector = embeddings.embed_query(doc.page_content)
    vectorstore.add_documents(documents=[doc], embedding=embeddings)

100%|██████████| 5/5 [00:01<00:00,  2.84it/s]


In [187]:
from langchain.vectorstores import Pinecone  # Import from langchain_pinecone
# ... (other imports)

# Instantiate the PineconeVectorStore with the correct arguments
retriever = PineconeVectorStore(
    index=index,  # Your Pinecone index object
    embedding=embeddings  # Your embedding function
)

In [188]:
!pip install -Uq langchain-google-genai

In [190]:
from langchain_google_genai import ChatGoogleGenerativeAI # Import ChatGoogleGenerativeAI instead of GoogleGeminiFlash

gemini_model = ChatGoogleGenerativeAI(
    model="gemini-pro",  # Example: using "gemini-pro" model
    google_gemini_api_key = 'GOOGLE_API_KEY',
    temperature=0.7
)

In [192]:
from langchain_pinecone import Pinecone

# Update retriever to use Pinecone class from langchain_pinecone
# Notice the changed class name: Pinecone -> PineconeVectorStore
retriever = PineconeVectorStore(index=index, embedding=embeddings)

from langchain.chains import RetrievalQA
retriever = vector_store.as_retriever() # Get retriever from vector_store

qa_chain = RetrievalQA.from_chain_type(
    llm=gemini_model,
    chain_type="stuff",  # Other options: "map_reduce", "refine"
    retriever=retriever
)

In [199]:
from langchain.chains import LLMChain
chain = LLMChain(llm=gemini_model, prompt=prompt_template)
question = "What were the major milestones in AI development during the 20th century?"
response = chain.run(question)
print(response)

**1943:** Warren McCulloch and Walter Pitts develop a mathematical model of a neuron, providing a foundation for understanding artificial neural networks.

**1950:** Alan Turing publishes "Computing Machinery and Intelligence," proposing the Turing Test as a measure of machine intelligence.

**1956:** The Dartmouth Summer Research Project on Artificial Intelligence is held, coining the term "artificial intelligence."

**1957:** Frank Rosenblatt develops the perceptron, an early artificial neural network.

**1965:** Joseph Weizenbaum creates ELIZA, an early natural language processing program.

**1966:** Nils Nilsson develops STRIPS, a planning system that enables AI agents to reason about actions and their consequences.

**1969:** Marvin Minsky and Seymour Papert publish "Perceptrons," which proves that perceptrons have limited computational power.

**1971:** John McCarthy and Nils Nilsson develop the Situation Calculus, a formal language for representing and reasoning about changing s